# Ecommerce A/B Testing


In [2]:
import os
import pandas as pd

if "KAGGLE_KERNEL_RUN_TYPE" in os.environ:
    DATA_PATH = "/kaggle/input/ecommerce-ab-testing-2022-dataset1/ecommerce_ab_testing_2022_dataset1"
else:
    DATA_PATH = "data/ecommerce_ab_testing_2022_dataset1"

ab_data = pd.read_csv(f'{DATA_PATH}/ab_data.csv')
ab_data.head()

,user_id,timestamp,group,landing_page,converted
0,851104,11:48.6,control,old_page,0
1,804228,01:45.2,control,old_page,0
2,661590,55:06.2,treatment,new_page,0
3,853541,28:03.1,treatment,new_page,0
4,864975,52:26.2,control,old_page,1
